In [1]:
import os
os.chdir('../')
%pwd

'e:\\code\\project\\Text-Summarizer'

Creating Entity

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

Configuration Manager

In [3]:
from textSummarizer.utils import read_yaml,create_directories 
from textSummarizer import constants

class ConfigurationManager:      #class to read the config file and params file
    def __init__(
            self,
            config_file_path = constants.CONFIG_FILE_PATH,       #constants is a file which contains all the paths 
            params_file_path = constants.PARAMS_FILE_PATH
            ):
         
        self.config = read_yaml(config_file_path)      #read_yaml is a function which reads the yaml file and returns a dictionary
        self.params = read_yaml(params_file_path)

        #config is a ConfigBox(shown in trials.ipynb) object which is a dictionary with dot notation access with help of read_yaml function
        create_directories([self.config.artifacts_root])              #create_directories is a function which creates directories if they don't exist; artifacts_root is the path where all the artifacts will be stored present in the config.yaml file


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )


Creating Components

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_metric, load_from_disk, load_dataset
import pandas as pd
import torch
import tqdm as tqdm

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        '''
        Splitting the dataset into smaller batches such that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements.
        '''
        for i in range(0, len(list_of_elements), batch_size):       #this finction is used to split the dataset into smaller batches
            yield list_of_elements[i:i + batch_size]       #yield is used to return a generator object -> generator object is an iterator which can be iterated only once


    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,
                                    batch_size=16,
                                    device='cuda'if torch.cuda.is_available() else 'cpu',
                                    column_text = 'article',
                                    column_summary = 'highlights'):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))

        for article_batch,target_batch in tqdm(
            zip(article_batches,target_batches),
            total=len(article_batches)):

            inputs = tokenizer(article_batch,max_length= 1024,
                               truncation=True, padding='max_length',return_tensors='pt')
            
            summaries = model.generate(input_ids= inputs['input_ids'].to(device),
                                       attention_mask = inputs['attention_mask'].to(device),           #attention_mask is a mask that is used to avoid performing attention on padding token indices 
                                       length_penalty=0.8,     # parameter for length penalty ensures that the model does not generate sequences that are too long. 
                                       num_beams=8,        #num_beams is the number of beams for beam search 
                                       max_length=128)     #max_length is the maximum length of the sequence to be generated. An input sequence longer than max_length will be truncated to max_length
            
            #We decode the genrated texts using the tokenizer, replace the token and add the decoded texts with references to metric

            decoded_summaries = [tokenizer.decode(g,skip_special_tokens=True,
                                                  clean_up_tokenization_spaces=True)
                                  for g in summaries]
            
            decoded_summaries = [d.replace('', " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries,references=target_batch)

        #Finally we compute the metric and return the ROUGE scores
        score = metric.compute()
        return score
    
    def evaluate(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        #Loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(dataset_samsum_pt['test'][0:10],
                                                 rouge_metric,model_pegasus,
                                                 tokenizer, batch_size=2,
                                                 column_text = 'dialogue', column_summary= 'summary')
        
        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df =pd.DataFrame([rouge_dict],index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

Creating Pipeline

In [ ]:
try:
    config = ConfigurationManager().get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=config)
    model_evaluation.evaluate()

except Exception as e:
    raise e